# 텐서플로우
## 텐서 플로우 설치 확인

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()  # 텐서플로우 2에 1을 적용하게끔 만드는 코드

In [ ]:
import tensorflow
tensorflow.__version__

In [ ]:
hello = tf.constant("Hello Tensorflow")

In [ ]:
hello

In [ ]:
session = tf.Session()
print(session.run(hello))

## 인공신경망 예제 (keras, MLPClassifier)

In [ ]:
from tensorflow.keras.datasets import mnist
(train_img, train_labels), (test_img, test_labels) = mnist.load_data()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

network = Sequential()
network.add(Dense(512, activation='relu', input_shape = (28*28,)))  #입력층: input_shape
network.add(Dense(10, activation='softmax'))


In [ ]:
network.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
#X의 모양을 입력 모양에 맞도록 수정

train_img = train_img.reshape((-1, 28*28))
test_img = test_img.reshape((10000, 28*28))

#표준화
train_img = train_img.astype('float32')/255
test_img = test_img.astype('float32') / 255

In [ ]:
#y를 원-핫 인코딩 

from tensorflow.keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
network.fit(train_img, train_labels, epochs=50, batch_size=128)

#128개 씩 가중치가 업데이트됨
# 총 5번 실행된

In [ ]:
test_loss, test_acc = network.evaluate(test_img, test_labels)

In [ ]:
test_acc

In [ ]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(hidden_layer_sizes=(512,), max_iter=5, batch_size=128, solver='adam')

In [ ]:
from tensorflow.keras.datasets import mnist
(train_img, train_labels), (test_img, test_labels) = mnist.load_data()

train_img = train_img.reshape((-1, 28*28))
test_img = test_img.reshape((10000, 28*28))

#원-핫 인코딩 하지 않음

In [ ]:
import numpy as np

model.fit(train_img, train_labels)

In [ ]:
model.score(test_img, test_labels)

## iris data
### 사이킷런을 이용한 인공신경망

In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets

iris = datasets.load_iris() #target이 종속 변수, data가 독립변수

iris_X = iris.data
iris_y = iris.target

In [ ]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(iris_X, iris_y, test_size=0.3, random_state=1)

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(50, 50, 30), activation='relu', verbose=True)

In [ ]:
mlp.fit(train_X, train_y)

In [ ]:
mlp.score(test_X, test_y)

### 텐서플로우를 이용한 인공신경망

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
import pandas as pd

iris_y = pd.get_dummies(iris.target).to_numpy() # 원핫 인코딩 후 넘파이 배열로 바꾸기 

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(iris_X, iris_y,test_size=0.3, random_state=1)

In [ ]:
# Tensor, Variable(W, b를 정의할 때), placeholder
# h(출력값) = f(w*x + b)

x = tf.placeholder(tf.float32, [None, 4])
y = tf.placeholder(tf.float32, [None, 3])
w = tf.Variable(tf.zeros([4,3])) #4행 3열을 0으로 채워주세요
b = tf.Variable(tf.zeros([3]))

In [ ]:
h= tf.nn.softmax(tf.matmul(x, w) +b) # 행렬의 곱(matmul)  >> h 출력

In [ ]:
cross_entropy = -tf.reduce_sum(y * tf.log(h), reduction_indices=[1])  # reduction_indices는 axis와 동일함

In [ ]:
loss = tf.reduce_mean(cross_entropy)

In [ ]:
# loss 최소화
train = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) #operation 실행

In [ ]:
for i in range(100000):
    _, loss_value = sess.run([train, loss], feed_dict={x: train_X, y:train_y})
    
    if i% 10000 == 0:
        print(f"{i}, {loss_value}")

In [ ]:
#예측
pred = sess.run(h, feed_dict={x:test_X})

In [ ]:
np.argmax(pred, axis=1)

In [ ]:
np.argmax(test_y, axis=1)

In [ ]:
pd.crosstab(np.argmax(test_y, axis=1), np.argmax(pred, axis=1))

### 케라스를 이용한 인공신경망 모형

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense

In [ ]:
train_X.shape, train_y.shape

In [ ]:
network = Sequential()

In [ ]:
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.layers import ELU, PReLU
from tensorflow.keras.initializers import RandomNormal

rand_init = RandomNormal(mean=0, stddev=0.1)
network.add(Input(4))
network.add(Dense(50, activation='sigmoid', kernel_initializer='he_normal'))
network.add(Dense(50, activation=ELU(alpha=2)))
network.add(Dense(30, activation=PReLU(), kernel_initializer=rand_init))
network.add(Dense(3, activation='softmax'))

In [ ]:
#기본


# network.add(Input(4))
# network.add(Dense(50, activation='relu'))
# network.add(Dense(50, activation='relu'))
# network.add(Dense(30, activation='relu'))
# network.add(Dense(3, activation='softmax'))

In [ ]:
network.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'] )

In [ ]:
network.fit(train_X, train_y, batch_size=30, epochs=200, verbose=1)

In [ ]:
network.evaluate(test_X, test_y)

In [ ]:
pred = np.argmax(network.predict(test_X), axis=1)

In [ ]:
pd.crosstab(np.argmax(test_y, axis=1), pred)

## 와인 데이터셋 분류

 
 - 레이어5개
 - 뉴런의 수 64, 128, 128, 64, 32
 - 활성화 함수 relu, elu, preu, sigmoid, relu
 - Weight 초기화 1~3 glorot_uniform, 4평균 0 표준편차 0.1인 난수 5 he_uniform
 - 손실함수 크로스엔트로피
 - 옵티마이저 Adam, 학습률 0.001, 배치 크기50, 학습횟수 200

In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets

redwine = pd.read_csv('winequality-red.csv', sep=';')

In [ ]:
redwine.quality.value_counts()

In [ ]:
redwine_X = redwine.iloc[:, :-1].to_numpy()
redwine_y = pd.get_dummies(redwine.iloc[:, -1]).to_numpy()

In [ ]:
redwine_y

In [ ]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(redwine_X, redwine_y, test_size=0.3, random_state=3)


In [ ]:
train_X.shape, train_y.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.layers import ELU, PReLU
from tensorflow.keras.initializers import RandomNormal

rand_init = RandomNormal(mean=0, stddev=0.1)

network = Sequential()

network.add(Input(11))
network.add(Dense(64, activation='relu', kernel_initializer='glorot_uniform'))
network.add(Dense(128, activation=ELU(alpha=0.1), kernel_initializer='glorot_uniform'))
network.add(Dense(256, activation=PReLU(), kernel_initializer='glorot_uniform'))
network.add(Dense(128, activation='sigmoid', kernel_initializer=rand_init))
network.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
network.add(Dense(6, activation='softmax'))

In [ ]:
from tensorflow.keras.optimizers import Adam

network.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

In [ ]:
network.fit(train_X, train_y, batch_size=100, epochs=300, verbose=1)

In [ ]:
network.evaluate(test_X, test_y)

In [ ]:
pred = np.argmax(network.predict(test_X), axis=1)

In [ ]:
pd.crosstab(np.argmax(test_y, axis=1)+3, pred+3)

In [ ]:
network.summary()

## 필기체 이미지 분류

### 케라스 Mnist

In [ ]:
from tensorflow.keras.datasets import mnist
(train_img, train_labels), (test_img, test_labels) = mnist.load_data()

In [ ]:
#X의 모양을 입력 모양에 맞도록 수정

train_img = train_img.reshape((-1, 28*28))
test_img = test_img.reshape((10000, 28*28))

#표준화
train_img = train_img.astype('float32')/255
test_img = test_img.astype('float32') / 255

#y를 원-핫 인코딩 

from tensorflow.keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

model = Sequential()

model.add(Dense(512, activation='relu', kernel_initializer='glorot_uniform', input_shape = (28*28,)))
model.add(Dense(256, activation='relu', kernel_initializer='glorot_uniform'))
model.add(Dense(218, activation='relu', kernel_initializer='glorot_uniform'))
model.add(Dense(10, activation='softmax'))

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(loss='categorical_crossentropy', optimizer=Adam(0.0001), metrics=['acc'])

In [ ]:
model.fit(train_img, train_labels, batch_size=200, epochs=50, verbose=1)

In [ ]:
model.evaluate(test_img, test_labels)

In [ ]:
pred = np.argmax(model.predict(test_img), axis=1)

In [ ]:
import pandas as pd

pd.crosstab(np.argmax(test_labels, axis=1), pred)

### 사이킷런을 이용한 인공신경망 모형

In [ ]:
import cv2
import numpy as np

img = cv2.imread('./images/digits.png', cv2.IMREAD_GRAYSCALE)

print(img.shape)

In [ ]:
cells = [np.hsplit(row, 100) for row in np.vsplit(img, 50)]
x=np.array(cells)

X = x[:,:].reshape(-1, 400).astype(np.float32)
y = np.repeat(np.arange(10), 500)

In [ ]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(512, 256, 128))


In [ ]:
mlp.fit(train_X, train_y)

In [ ]:
mlp.score(test_X, test_y)

### 텐서플로우를 이용한 인공신경망 모형

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd
import cv2

img = cv2.imread('./images/digits.png', cv2.IMREAD_GRAYSCALE)

print(img.shape)

Instructions for updating:
non-resource variables are not supported in the long term
(1000, 2000)


In [2]:
cells = [np.hsplit(row, 100) for row in np.vsplit(img, 50)]
x=np.array(cells) /255.

X = x[:,:].reshape(-1, 400).astype(np.float32)
y = pd.get_dummies(np.repeat(np.arange(10), 500)).to_numpy()

In [3]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=1)


In [4]:
X.shape, y.shape

((5000, 400), (5000, 10))

In [5]:
x = tf.placeholder(tf.float32, [None, 400])
y = tf.placeholder(tf.float32, [None, 10])  #None인 이유는 얼마를 넣을 지 모름

In [6]:
w1 = tf.Variable(tf.truncated_normal([400, 512], stddev=0.1))
b1 = tf.Variable(tf.constant(0.1, shape=[512]))
h1 = tf.nn.relu(tf.matmul(x, w1) + b1)


In [7]:
w2 = tf.Variable(tf.truncated_normal([512, 256], stddev=0.1))
b2 = tf.Variable(tf.constant(0.1, shape=[256]))
h2 = tf.nn.relu(tf.matmul(h1, w2) + b2)

In [8]:
w3 = tf.Variable(tf.truncated_normal([256, 128], stddev=0.1))
b3 = tf.Variable(tf.constant(0.1, shape=[128]))
h3 = tf.nn.relu(tf.matmul(h2, w3) + b3)

In [9]:
W = tf.Variable(tf.truncated_normal([128, 10], stddev=0.1))
b = tf.Variable(tf.constant(0.1, shape=[10]))

In [10]:
h = tf.nn.softmax(tf.matmul(h3, W) + b)

In [11]:
cross_entropy = -tf.reduce_sum(y * tf.log(h), reduction_indices=[1])

In [12]:
loss = tf.reduce_mean(cross_entropy)

train = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

In [13]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [14]:
tot_cnt = len(train_X)

for epoch in range(1000):
    avg_loss = 0
    batch_size=128
    tot_iter = int(tot_cnt/batch_size)
    for step in range(tot_iter):
        batch_X = train_X[int(step*batch_size): int((step+1)*batch_size)]
        batch_y = train_y[int(step*batch_size): int((step+1)*batch_size)]
        _, loss_ = sess.run([train, loss], feed_dict={x:batch_X, y:batch_y})
        avg_loss += (loss_/tot_iter)
        print("{:03d} - {:f}".format(step, loss_), end="\r")
    if tot_cnt%tot_iter !=0:
        batch_X = train_X[int(tot_iter*batch_size):]
        batch_y = train_y[int(tot_iter*batch_size):]
        _, loss_ = sess.run([train, loss], feed_dict={x:batch_X, y:batch_y})
        avg_loss += (loss_/tot_iter)
        print("{:03d} - {:f}".format(step, loss_), end="\r")
    if epoch % 100==0:
        print("step {}, train loss {}".format(epoch, avg_loss))

step 0, train loss 1.7277011076609292
step 100, train loss 0.005881198017892462
step 200, train loss 0.0017884826983963313
step 300, train loss 0.0009741806042492735
step 400, train loss 0.0006485185828447964
step 500, train loss 0.0004776148144392884
step 600, train loss 0.00037400447736554606
step 700, train loss 0.0003051331594987358
step 800, train loss 0.00025624070110131505
step 900, train loss 0.00021997318415540375


In [16]:
correct_pred = tf.equal(tf.argmax(h, 1), tf.argmax(y,1))
acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
print(sess.run(acc, feed_dict={x:test_X, y:test_y}))

0.94133335


In [20]:
pred_y = sess.run(tf.argmax(h,1), feed_dict={x:test_X})
cross_tab = pd.crosstab(test_y.argmax(axis=1), pred_y,margins=True)

In [21]:
cross_tab

col_0,0,1,2,3,4,5,6,7,8,9,All
row_0,,,,,,,,,,,
0,152,0,0,0,0,1,0,2,0,0,155
1,0,148,0,0,0,1,0,0,0,0,149
2,1,2,138,2,3,1,0,4,1,0,152
3,0,3,2,161,0,2,0,2,1,0,171
4,0,0,3,0,121,0,0,0,1,5,130
5,0,1,2,3,1,138,2,0,0,1,148
6,1,0,0,0,1,3,130,0,0,0,135
7,1,2,1,2,1,1,0,151,0,1,160
8,0,2,1,2,2,3,1,0,144,1,156
